Напиши мне, если что-то не так или есть способы сделать это круче 'amir147@rambler.ru'

In [2]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import StratifiedKFold, cross_val_score
from sklearn.preprocessing import scale
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

In [4]:
from Blending import Blender, BaseProbaEstimator
from Stacking import Stacker

In [5]:
data = pd.read_csv('log_data.csv', header=None)

## Baselines

#### LR

In [6]:
for i in [3, 7, 43]:
    sskkff = StratifiedKFold(data[0], n_folds=10, shuffle=True, random_state=i )
    roc_auc = []
    for train_idx, test_idx in sskkff:
        
        train = data.loc[train_idx]
        labels = train[0].values
        test = data.loc[test_idx]
        labels_ts = test[0].values
        train = train.drop(0, axis=1).values
        test = test.drop(0, axis=1).values
        
        lr = LogisticRegression(n_jobs=-1)
        lr.fit(scale(train), labels)
        ans = lr.predict_proba(scale(test))

        roc_auc.append(roc_auc_score(labels_ts, ans[:, 1]))
    print (np.mean(roc_auc), '±', np.std(roc_auc))
    print('one_rs', i)
    


0.929272727273 ± 0.0611768875495
one_rs 3
0.932181818182 ± 0.0825534637684
one_rs 7
0.936454545455 ± 0.067323629391
one_rs 43


#### KNN

In [7]:
for i in [3, 7, 43]:
    sskkff = StratifiedKFold(data[0], n_folds=10, shuffle=True, random_state=i )
    roc_auc = []
    for train_idx, test_idx in sskkff:
        
        train = data.loc[train_idx]
        labels = train[0].values
        test = data.loc[test_idx]
        labels_ts = test[0].values
        train = train.drop(0, axis=1).values
        test = test.drop(0, axis=1).values
        
        knn = KNeighborsClassifier(n_jobs=-1)
        knn.fit(scale(train), labels)
        ans = knn.predict_proba(scale(test))

        roc_auc.append(roc_auc_score(labels_ts, ans[:, 1]))
    print (np.mean(roc_auc), '±', np.std(roc_auc))
    print('one_rs', i)
    


0.954681818182 ± 0.0552806700334
one_rs 3
0.915454545455 ± 0.114301327151
one_rs 7
0.961454545455 ± 0.0587440167518
one_rs 43


#### RF

In [8]:
for i in [3, 7, 43]:
    sskkff = StratifiedKFold(data[0], n_folds=10, shuffle=True, random_state=i )
    roc_auc = []
    for train_idx, test_idx in sskkff:
        
        train = data.loc[train_idx]
        labels = train[0].values
        test = data.loc[test_idx]
        labels_ts = test[0].values
        train = train.drop(0, axis=1).values
        test = test.drop(0, axis=1).values
        
        rf = RandomForestClassifier( random_state= 0 , n_jobs=-1)
        rf.fit(scale(train), labels)
        ans = rf.predict_proba(scale(test))

        roc_auc.append(roc_auc_score(labels_ts, ans[:, 1]))
    print (np.mean(roc_auc), '±', np.std(roc_auc))
    print('one_rs', i)
    



0.9365 ± 0.043993355058
one_rs 3
0.899409090909 ± 0.109073303504
one_rs 7
0.946181818182 ± 0.0658548993515
one_rs 43


#### GradBoost

In [9]:
for i in [3, 7, 43]:
    sskkff = StratifiedKFold(data[0], n_folds=10, shuffle=True, random_state=i )
    roc_auc = []
    for train_idx, test_idx in sskkff:
        
        train = data.loc[train_idx]
        labels = train[0].values
        test = data.loc[test_idx]
        labels_ts = test[0].values
        train = train.drop(0, axis=1).values
        test = test.drop(0, axis=1).values
        
        gb = GradientBoostingClassifier(random_state= 0)
        gb.fit(scale(train), labels)
        ans = gb.predict_proba(scale(test))

        roc_auc.append(roc_auc_score(labels_ts, ans[:, 1]))
    print (np.mean(roc_auc), '±', np.std(roc_auc))
    print('one_rs', i)
    


0.947045454545 ± 0.0494269851347
one_rs 3
0.923090909091 ± 0.0897555909997
one_rs 7
0.964181818182 ± 0.0356502755776
one_rs 43


_____

## Blending (вернее стекинг если не задаем веса - решается логлосс задача с огр. ) 

In [ ]:
'''
#( классификаторы с интерфейсом BaseEstimator)
________________________________________
| Первый уровень | второй уровень       |
|________________|______________________|
|      LR        |                      |
|      RF        |0.4*lr+0.3*rf+0.3*xgb | тут веса подбираются минимизацией кросс-энтропийного функционала 
|  XGBClassifier |                      |     или задаем свои
|________________|______________________|
'''

In [10]:
# blend of vanilla classifiers with optimization inside

for i in [3, 7, 43]:
    sskkff = StratifiedKFold(data[0], n_folds=10, shuffle=True, random_state=i )
    roc_auc = []
    for train_idx, test_idx in sskkff:
        
        train = data.loc[train_idx]
        labels = train[0].values
        test = data.loc[test_idx]
        labels_ts = test[0].values
        train = train.drop(0, axis=1).values
        test = test.drop(0, axis=1).values
        
        st = Blender([GradientBoostingClassifier(random_state=0), 
              KNeighborsClassifier(n_jobs=-1), 
              LogisticRegression(n_jobs=-1), 
              RandomForestClassifier(random_state=0, n_jobs=-1)
             ])
        
        st.fit(scale(train), labels)
        ans = st.predict_proba( scale(test) )
        roc_auc.append(roc_auc_score(labels_ts, ans[:,1]))
    print (np.mean(roc_auc), '±', np.std(roc_auc))
    print('one_rs', i)
    

0.957909090909 ± 0.0450363489558
one_rs 3
0.939181818182 ± 0.0765392461313
one_rs 7
0.957818181818 ± 0.0621836788664
one_rs 43


In [11]:
# vanilla blend with our weights 
# Ex. -> Only log_legression compare with results above (in baselines section)


for i in [3, 7, 43]:
    sskkff = StratifiedKFold(data[0], n_folds=10, shuffle=True, random_state=i )
    roc_auc = []
    for train_idx, test_idx in sskkff:
        
        train = data.loc[train_idx]
        labels = train[0].values
        test = data.loc[test_idx]
        labels_ts = test[0].values
        train = train.drop(0, axis=1).values
        test = test.drop(0, axis=1).values
        
        st = Blender([GradientBoostingClassifier( random_state=0), 
              KNeighborsClassifier( n_jobs=-1), 
              LogisticRegression( n_jobs=-1)], with_own_weights = True)
        
        st.fit(scale(train), labels)
        weights = np.array([0.0, 0.0, 1.0]) #sum(weights)==1 
        ans = st.predict_proba( scale(test), weights = weights )
        roc_auc.append(roc_auc_score(labels_ts, ans[:, 1]))
    print (np.mean(roc_auc), '±', np.std(roc_auc))
    print('one_rs', i)


0.929272727273 ± 0.0611768875495
one_rs 3
0.932181818182 ± 0.0825534637684
one_rs 7
0.936454545455 ± 0.067323629391
one_rs 43


_____

## Stacking

In [ ]:
'''
#( классификаторы с интерфейсом BaseEstimator)
__________________________________________________________ 
| Первый уровень | второй уровень | ы | последний уровень |
|________________|________________|___|___________________|
|      LR        | GradientBoost  |.. |                   |
|      RF        |       KNN      |.. |        LR         |
|  XGBClassifier |       LR       |.. |                   |
|________________|________________|___|___________________|
'''

In [12]:
# vanilla 
for i in [3, 7, 43]:
    sskkff = StratifiedKFold(data[0], n_folds=10, shuffle=True, random_state=i )
    roc_auc = []
    for train_idx, test_idx in sskkff:
        
        train = data.loc[train_idx]
        labels = train[0].values
        test = data.loc[test_idx]
        labels_ts = test[0].values
        train = train.drop(0, axis=1).values
        test = test.drop(0, axis=1).values
        
        st = Stacker([
               
                [
                 GradientBoostingClassifier( random_state=0), 
                 KNeighborsClassifier( n_jobs=-1), 
                 LogisticRegression(n_jobs=-1),
                 RandomForestClassifier(n_jobs=-1, random_state=0)
                ],
                [LogisticRegression(C = 1e-1, n_jobs=-1)]
            ])
        
        st.fit(scale(train), labels)
        ans = st.predict_proba(scale(test))
        
        roc_auc.append(roc_auc_score(labels_ts, ans[:, 1]))
    print (np.mean(roc_auc), '±', np.std(roc_auc))
    print('one_rs', i)

0.948545454545 ± 0.047048770195
one_rs 3
0.934363636364 ± 0.0760891169551
one_rs 7
0.964363636364 ± 0.0473108238101
one_rs 43


In [13]:
# random params

for i in [3, 7, 43]:
    sskkff = StratifiedKFold(data[0], n_folds=10, shuffle=True, random_state=i )
    roc_auc = []
    for train_idx, test_idx in sskkff:
        
        train = data.loc[train_idx]
        labels = train[0].values
        test = data.loc[test_idx]
        labels_ts = test[0].values
        train = train.drop(0, axis=1).values
        test = test.drop(0, axis=1).values
        
        st = Stacker([
                [
                     LogisticRegression(C=1e-2, n_jobs=-1), 
                     GradientBoostingClassifier(n_estimators = 50, learning_rate=0.09, random_state=0),
                ],
                [
                     RandomForestClassifier(n_estimators=120, n_jobs=-1, random_state=0),
                     KNeighborsClassifier(n_neighbors = 1, n_jobs=-1), 
                     LogisticRegression(C=1e-2, n_jobs=-1)
                ],
                [LogisticRegression(C = 10, n_jobs=-1)]
            ])
        
        st.fit(scale(train), labels)
        ans = st.predict_proba(scale(test))
        
        roc_auc.append(roc_auc_score(labels_ts, ans[:, 1]))
    print (np.mean(roc_auc), '±', np.std(roc_auc))
    print('one_rs', i)

0.962818181818 ± 0.0368343398888
one_rs 3
0.954363636364 ± 0.0724526633724
one_rs 7
0.976454545455 ± 0.0315742598377
one_rs 43
